In [1]:
import random
import numpy as np
from deap import base, creator, tools

# Define the size of the population, number of generations, and other parameters
POP_SIZE = 100
NUM_GENERATIONS = 50
MUTATION_PROBABILITY = 0.1

# Define the size of the input matrix (e.g., a stoichiometric matrix)
MATRIX_SIZE = (10, 10)

# Define the fitness evaluation function
# This function takes an individual (a matrix) as input and returns its fitness value
# Fitness is typically defined as the objective function to optimize in a genetic algorithm
def evaluate(matrix):
    # Example fitness function: sum of all elements in the matrix
    return np.sum(matrix),

# Define the mutation operation for an individual (a matrix)
# This function takes an individual as input and mutates it
# Mutation can be implemented as randomly modifying the matrix
def mutate(matrix):
    # Example mutation: randomly modify one element in the matrix
    i = random.randint(0, MATRIX_SIZE[0] - 1)
    j = random.randint(0, MATRIX_SIZE[1] - 1)
    matrix[i][j] = random.random()  # Replace with your own mutation operation

# Create the individual class and population class using DEAP
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize the fitness value
creator.create("Individual", np.ndarray, fitness=creator.FitnessMin)

# Register the necessary functions and classes in the DEAP toolbox
toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=MATRIX_SIZE[0] * MATRIX_SIZE[1])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Create the initial population
population = toolbox.population(n=POP_SIZE)

# Run the genetic algorithm
for gen in range(NUM_GENERATIONS):
    # Select the next generation
    offspring = toolbox.select(population, len(population))
    offspring = list(offspring)

    # Clone the selected individuals
    offspring_clone = [toolbox.clone(ind) for ind in offspring]

    # Apply crossover and mutation to the offspring
    for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < 0.5:
            toolbox.mate(ind1, ind2)
            del ind1.fitness.values
            del ind2.fitness.values

    for ind in offspring:
        if random.random() < MUTATION_PROBABILITY:
            toolbox.mutate(ind)
            del ind.fitness.values

    # Evaluate the fitness of the offspring
    fitnesses = toolbox.map(toolbox.evaluate, offspring_clone)
    for ind, fit in zip(offspring_clone, fitnesses):
        ind.fitness.values = fit

    # Replace the old population with the offspring
    population[:] = offspring_clone

# Get the best individual from the final population
best_individual = tools.selBest(population, 1)[0]
best_matrix = np.reshape(best_individual, MATRIX_SIZE)

# Print the best matrix and its fitness value
print("Best matrix")


TypeError: 'numpy.float64' object does not support item assignment

In [2]:
import random

print(random.random())

0.7617479013166362


In [7]:
matrix = np.random.rand(5, 5)
matrix

array([[0.46512305, 0.66527745, 0.35893896, 0.69745698, 0.84783487],
       [0.884484  , 0.04218397, 0.4455238 , 0.25161307, 0.71954831],
       [0.8643763 , 0.78332311, 0.75795632, 0.24241287, 0.08367897],
       [0.52738094, 0.38734627, 0.72402238, 0.47580397, 0.4026507 ],
       [0.1871806 , 0.56728277, 0.25227319, 0.42587065, 0.34111874]])

In [8]:
MATRIX_SIZE = (10, 10)
# Example mutation: randomly modify one element in the matrix
i = random.randint(0, MATRIX_SIZE[0] - 1)
j = random.randint(0, MATRIX_SIZE[1] - 1)
matrix[i][j] = random.random()  # Replace with your own mutation operation

In [9]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

# Define the problem

# Define the fitness function
def evaluate(individual, matrix):
    # individual: a binary string representing the genes of an individual
    # matrix: the input matrix for the problem
    # Perform some computation on the matrix using the individual
    # Return a fitness value based on the computation result
    # In this example, we'll just return the sum of the values in the matrix that correspond to the selected genes in the individual
    selected_genes = np.where(individual)[0]
    fitness = np.sum(matrix[:, selected_genes])
    return fitness,

# Define the genetic algorithm parameters
POP_SIZE = 100
NUM_GENERATIONS = 50
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.8
TOURNAMENT_SIZE = 3

# Create the input matrix (example matrix of size 5x5 with random values)
matrix = np.random.rand(5, 5)

# Create a binary string representation of the matrix for the individuals
IND_SIZE = matrix.shape[1]
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("genes", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.genes, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=TOURNAMENT_SIZE)
toolbox.register("evaluate", evaluate, matrix=matrix)

# Create the initial population
population = toolbox.population(n=POP_SIZE)

# Run the genetic algorithm
for gen in range(NUM_GENERATIONS):
    # Evaluate the fitness of each individual in the population
    fitnesses = map(toolbox.evaluate, population)
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    # Select the next generation of individuals
    offspring = toolbox.select(population, len(population))
    offspring = list(offspring)

    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover and mutation to the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CROSSOVER_RATE:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTATION_RATE:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Replace the old population with the offspring
    population[:len(offspring)] = offspring

# Get the best individual in the final population
best_individual = tools.selBest(population, 1)[0]

# Print the best individual and its fitness value
print("Best Individual:", best_individual)
print("Best Fitness Value:", best_individual.fitness.values[0])


Best Individual: [1 1 1 1 1]
Best Fitness Value: 14.02529195993951


C:\Users\rodri\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
